# Laboratorium 1 - content-based recommender

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas

from sklearn.model_selection import train_test_split, KFold

In [2]:
# tworzymy reprezentacje filmow jako wektorow cech - na podstawie gatunkow

genres = [
    '(no genres listed)', 
    'Action', 
    'Adventure', 
    'Animation', 
    'Children', 
    'Comedy', 
    'Crime', 
    'Documentary', 
    'Drama', 
    'Fantasy', 
    'Film-Noir', 
    'Horror', 
    'IMAX', 
    'Musical', 
    'Mystery', 
    'Romance', 
    'Sci-Fi', 
    'Thriller', 
    'War', 
    'Western'
]
genres_no = len(genres)

movies = pandas.read_csv('ml-latest-small/movies.csv')
movies_no = movies.shape[0]

movies['bias'] = 1.0
for genre in genres:
    movies[genre] = np.where(movies['genres'].str.contains(genre, regex=False), 1.0, 0.0)
    
movies = movies.drop(columns=['title', 'genres']).set_index('movieId')
movies

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# wczytujemy oceny uytkownikow i od razu dzielimy je na dwa zbiory - treningowy i testowy

all_ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
train_ratings_set, test_ratings_set = train_test_split(all_ratings, test_size=0.05)
train_ratings_set

,userId,movieId,rating
20467,135,1722,1.0
7738,51,7000,5.0
30481,212,134130,4.0
32999,225,1717,2.0
49642,318,81537,3.5
...,...,...,...
82602,524,1278,5.0
43104,288,39414,3.5
49080,318,1220,3.5
55994,370,39,3.5


In [4]:
# inicjalizujemy macierz preferencji uzytkownikow liczbami losowymi z przedzialu [0.0, 5.0]

def initialize_users(raw_ratings):
    users_no = raw_ratings['userId'].unique().size
    users = pandas.DataFrame(5.0 * np.random.uniform(size=(users_no, genres_no+1)), index=raw_ratings['userId'].unique(), columns=['bias']+genres)
    return users_no, users

users_no, users = initialize_users(train_ratings_set)
users

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
135,0.693574,4.929437,1.284975,2.737032,3.624019,2.476372,0.151063,4.911651,4.580159,3.307935,...,3.020166,3.587940,3.433542,0.725051,0.057649,1.293572,4.747346,1.471225,0.233705,1.801801
51,4.054156,0.769907,3.709764,4.973097,1.467441,3.264520,0.254491,1.419183,1.024481,4.142159,...,2.930831,2.333302,2.123288,0.893777,2.754482,2.932311,1.005153,3.008534,1.891564,3.506568
212,2.667909,4.587531,0.578196,4.146456,0.467832,2.474212,2.793302,1.568816,1.659572,4.618747,...,0.202849,1.640449,0.032565,3.528172,0.232382,3.235201,4.336657,1.249572,4.886803,2.374843
225,3.340591,0.774226,3.506110,1.969272,0.863856,2.979486,3.092214,4.781917,2.022935,1.673625,...,1.585422,3.070709,1.645284,0.505495,3.386916,0.545586,2.185344,2.781901,2.512153,1.445233
318,1.989991,2.004050,0.281931,0.206705,0.886606,1.878455,4.164619,2.042208,2.239344,0.231662,...,1.931236,4.826098,4.728761,0.303792,1.135179,1.119940,4.647953,1.322456,2.630794,1.160188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,3.677828,2.975472,1.375640,3.784289,1.414482,4.239119,1.318748,2.902220,2.670993,3.954879,...,1.639621,4.989988,3.672271,0.261581,1.591784,1.347772,3.661003,4.958996,2.337172,0.276615
574,3.117671,4.766531,2.642977,3.395950,3.698882,4.617742,2.536848,3.614459,1.920729,2.832186,...,3.469974,0.064047,0.898372,0.460989,3.262075,1.123864,0.285124,2.102173,2.140778,1.338266
496,4.160617,2.679425,2.205323,2.227482,0.976891,3.255083,3.656627,4.335175,1.801038,1.257221,...,0.002625,4.996484,3.596328,2.829225,3.564184,2.289180,4.811867,4.677762,0.429564,3.781182
535,0.960031,3.613721,0.974054,3.138268,3.881921,4.542510,3.797911,2.387653,0.747891,2.219766,...,0.392534,4.471327,3.914851,1.534156,0.333085,2.423690,3.030615,1.849005,3.827838,2.394881


In [5]:
# za pomoca sprytnej sztuczki przeksztalcamy oceny z formatu dostarczonego przez MovieLens do uzytecznej macierzy
# zwroc uwage na to, ze czesci filmow moze brakowac po podziale datasetu na dwie czesci - musimy uzueplnic brakujace kolumny
def get_ratings(raw_ratings, movies):
    ratings = raw_ratings.pivot(*raw_ratings.columns).fillna(0.0)
    missing_movies = set(movies.index).difference(set(raw_ratings['movieId']))
    for movie in missing_movies:
        ratings[movie] = 0.0
    ratings = ratings.reindex(sorted(ratings.columns), axis=1)
    return ratings

ratings = get_ratings(train_ratings_set, movies)
ratings

/var/folders/kc/cy9wh4850g9__x38vljlzyhh0000gn/T/ipykernel_22912/1522620778.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Część 2. - trening modelu

In [6]:
# trenujemy model iteracyjnie, wykorzystujac gradient descent

# learning speed
alpha = 0.00006 
# minimal upgrade for each step
delta = 100 
# regularization weight
lambd = 0.01 

def calculate_user_preferences(users, movies, ratings, raw_ratings, users_no, movies_no, alpha, delta, lambd):
    total_error = 0.0
    model = users

    while(True):
        previous_total_error = total_error
        # mozemy to policzyc jako iloczyn skalarny preferencji uzytkownikow i cech filmow
        predicted_ratings =  np.dot(model, movies.T)
        # tu stosujemy bardzo przydatna funkcje NumPy
        errors = np.where(ratings==0.0, pandas.DataFrame(np.zeros((users_no, movies_no))), predicted_ratings - ratings)
        # znow iloczyn skalarny - tym razem bledow
        gradient = np.dot(errors, movies)

        # tu stosujemy pewna sztuczke - rozbijamy sobie macierz z wyrazami regularyzujacymi na dwie
        # pierwsza to kolumna zlozona z zer
        regularization_k0 = pandas.DataFrame(np.zeros((users_no, 1)), index=raw_ratings['userId'].unique(), columns=['bias'])
        # druga to macierz preferencji uzytkownikow (czyli modelu) - bez pierwszej kolumny
        regularization_k = model.iloc[:,1:]
        # teraz sklejamy obie macierze
        regularization = pandas.concat([regularization_k0, regularization_k], axis=1)

        # najwazniejszy krok - aktualizacja modelu, czyli wszystkich wag
        model = model - alpha * (gradient + lambd * regularization)

        # suma wszystkich bledow
        total_error = np.sum(np.abs(errors))
        progress = abs(previous_total_error - total_error)
        print(total_error, progress)
        if progress < delta:
            break
            
    return model

prediction_model = calculate_user_preferences(users, movies, ratings, train_ratings_set, users_no, movies_no, alpha, delta, lambd)

583085.3704745437 583085.3704745437
546660.0337130075 36425.33676153619
516231.5340551392 30428.49965786829
490922.27230102324 25309.261754115927
469694.5519905482 21227.720310475037
451582.6780975768 18111.87389297143
435987.9912879622 15594.686809614592
422388.4612531771 13599.5300347851
410313.5151851387 12074.946068038407
399499.34513678105 10814.170048357628
389705.9792901785 9793.365846602537
380800.9287504322 8905.050539746298
372613.66906320915 8187.259687223064
365070.7731704854 7542.8958927237545
358110.47846074495 6960.294709740439
351636.90625589667 6473.572204848286
345599.25484202435 6037.651413872314
339932.6351521106 5666.619689913758
334597.0429617881 5335.592190322524
329584.097154157 5012.945807631069
324849.9073903171 4734.1897638398805
320373.2609155699 4476.64647474722
316137.5917178504 4235.669197719486
312115.32911770936 4022.2626001410536
308286.1092668789 3829.2198508304427
304638.3142111887 3647.7950556902215
301158.830816933 3479.483394255687
297841.51455429

156587.36247813315 218.76874106845935
156370.21110640047 217.1513717326743
156154.39527558727 215.81583081319695
155939.9544503294 214.4408252578869
155726.75446587158 213.19998445780948
155514.73377460954 212.02069126203423
155304.09697120276 210.63680340678548
155094.6608437566 209.4361274461553
154886.34235645784 208.3184872987622
154679.2291368861 207.1132195717364
154473.40835805048 205.82077883562306
154268.67168108516 204.7366769653163
154065.0378715331 203.6338095520623
153862.46274291846 202.57512861464056
153660.88599890092 201.57674401754048
153460.46521743553 200.4207814653928
153261.16624236142 199.29897507411079
153063.0245209771 198.14172138430877
152866.00903030045 197.01549067665474
152670.0439835442 195.96504675626056
152475.10132392426 194.9426596199337
152281.17858903974 193.9227348845161
152088.16597113718 193.0126179025683
151896.0126942717 192.153276865487
151704.82867243883 191.18402183285798
151514.64462715067 190.1840452881588
151325.39520389063 189.2494232600

## Część 3. - ocena jakości algorytmu

In [7]:
# na podstawie zbioru testowego i wytrenowanego modelu obliczamy metryki opisujace jakosc modelu

positive_threshold = 4.0
negative_threshold = 2.0

def calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold):
    # obliczamy true_positives itp.
    # nastepnie wszystkie metryki

    ratings = test_ratings_set.set_index(['userId', 'movieId'])
    predicted = predicted_ratings.stack().to_frame().loc[ratings.index.values].values.T[0]
    ratings = ratings.values.T[0]

    ratings_positive = np.where(ratings > positive_threshold,1,0)
    predicted_positive = np.where(predicted > positive_threshold,1,0)
    a = ratings_positive + predicted_positive
    true_positives = np.sum(a==2)
    
    ratings_negative = np.where(ratings < negative_threshold,1,0)
    predicted_negative = np.where(predicted < negative_threshold,1,0)
    a = ratings_negative + predicted_negative
    true_negatives = np.sum(a==2)
    
    ratings_positive = np.where(ratings > positive_threshold,-1,0)
    predicted_positive = np.where(predicted > positive_threshold,1,0)
    a = ratings_positive + predicted_positive
    false_positives = np.sum(a==1)
    
    ratings_negative = np.where(ratings < negative_threshold,-1,0)
    predicted_negative = np.where(predicted < negative_threshold,1,0)
    a = ratings_negative + predicted_negative
    false_negatives = np.sum(a==1)

    
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
    
    precision = true_positives / (true_positives + false_positives)
    
    recall = true_positives / (true_positives + false_negatives)
    
    f1 = 2 * recall * precision / (recall + precision)

    return {
        'true_positives': true_positives,
        'true_negatives': true_negatives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [8]:
predicted_ratings = prediction_model.dot(movies.T)
calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold)

{'true_positives': 562,
 'true_negatives': 69,
 'false_positives': 1920,
 'false_negatives': 771,
 'accuracy': 0.18994581577363034,
 'precision': 0.2264302981466559,
 'recall': 0.4216054013503376,
 'f1': 0.2946264744429882}

In [9]:
# dla porownania - obliczmy te same metryki dla modelu losowego
# zauwaz, w jaki sposob ponownie wykorzystujemy funkcje inicjalizujaca preferencje uzytkownikow

_, random_model = initialize_users(train_ratings_set)
random_prediction = random_model.dot(movies.T)
calculate_stats(test_ratings_set, random_prediction, positive_threshold, negative_threshold)

{'true_positives': 984,
 'true_negatives': 8,
 'false_positives': 3578,
 'false_negatives': 103,
 'accuracy': 0.21228332976674513,
 'precision': 0.21569487067075843,
 'recall': 0.9052437902483901,
 'f1': 0.348380244291025}

## Część 4. - istotność statystyczna

In [20]:
# wielokrotnie uruchamiamy trening modelu
# za każdym razem dzielimy dataset na zbior treningowy i testowy w inny sposob - klasa KFold robi to za nas
# zwroc uwage na bardzo istotny szczegol - oba modele, wytrenowany i losowy, musza byc porownywane na tym samym zbiorze testowym

n_tests = 5
results = []
random_results = []
raw_ratings = all_ratings

for train, test in KFold(n_splits=n_tests, shuffle=True).split(raw_ratings):
    test_set = raw_ratings.iloc[test]
    train_set = raw_ratings.iloc[train]
    
    # wygeneruj macierz użytkowników i ocen
    _, users = initialize_users(raw_ratings)
    ratings = get_ratings(train_set, movies)
    
    # wytrenuj model
    model = calculate_user_preferences(users, movies, ratings, train_set, users_no, movies_no, alpha, delta, lambd)
    
    # oblicz metryki dla wytrenowanego modelu
    predicted_ratings = model.dot(movies.T)
    stats_model = calculate_stats(test_set, predicted_ratings, positive_threshold, negative_threshold)
    results.append(stats_model)
    
    # oblicz metryki dla modelu losowego
    _, random_model = initialize_users(train_set)
    random_prediction = random_model.dot(movies.T)
    stats_random = calculate_stats(test_set, random_prediction, positive_threshold, negative_threshold)
    random_results.append(stats_random)

/var/folders/kc/cy9wh4850g9__x38vljlzyhh0000gn/T/ipykernel_22912/1522620778.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


477076.9283353265 477076.9283353265
455174.6121426915 21902.316192634986
436245.77245480113 18928.839687890373
419952.21478830464 16293.55766649649
405742.9843691332 14209.23041917145
393295.3745711902 12447.609797942976
382175.96819530294 11119.40637588728
372077.6644538145 10098.303741488431
362856.4264953359 9221.237958478625
354366.49940897647 8489.927086359414
346528.295526757 7838.2038822194445
339258.7967003431 7269.498826413939
332486.7587315903 6772.037968752789
326189.95861673943 6296.800114850863
320308.2432667637 5881.7153499757405
314810.6075206051 5497.635746158601
309661.74314522574 5148.864375379344
304817.5790305482 4844.164114677522
300257.87399514485 4559.705035403371
295947.21615080436 4310.657844340487
291862.82472328877 4084.3914275155985
287990.9430573052 3871.8816659835866
284311.558439809 3679.3846174961654
280798.17601089354 3513.3824289154727
277458.22751720325 3339.948493690288
274274.9432361542 3183.28428104904
271225.3551935976 3049.5880425565992
268301.92

139125.84902906875 200.89673777815187
138926.0827102663 199.76631880245986
138727.4108096179 198.67190064839087
138530.06925053804 197.34155907985405
138333.8558087271 196.213441810949
138138.7697221491 195.08608657799778
137944.77024448366 193.99947766543482
137751.85115803216 192.9190864514967
137560.0594407833 191.79171724885236
137369.59299908244 190.4664417008753
137180.15653598175 189.43646310069016
136991.86950146523 188.2870345165138
136804.57924546863 187.29025599660235
136618.27376971603 186.30547575259698
136432.88487150156 185.38889821447083
136248.50157355113 184.38329795043683
136065.14783832958 183.35373522154987
135882.89408813164 182.25375019793864
135701.603573336 181.2905147956335
135521.2699119617 180.33366137431585
135341.8129298092 179.4569821524783
135163.2959669398 178.51696286941296
134985.8442131313 177.4517538085056
134809.34650023043 176.4977129008621
134633.84087390653 175.50562632389483
134459.27985620536 174.56101770116948
134285.62948104634 173.650375159

199659.3131028603 933.7608129436849
198743.76109445206 915.5520084082382
197845.92576434326 897.8353301087918
196965.17987659018 880.745887753088
196100.39455475693 864.7853218332457
195250.58301282578 849.8115419311507
194416.83162429696 833.7513885288208
193598.0280570718 818.8035672251717
192794.71050990556 803.3175471662253
192005.99498620274 788.7155237028201
191230.70428211012 775.290704092622
190467.48106001684 763.2232220932783
189716.9335278693 750.5475321475533
188978.9978557808 737.9356720884971
188253.9653929471 725.0324628336821
187540.2849943591 713.6803985880106
186836.99226553374 703.2927288253559
186144.26685959255 692.7254059411935
185462.86121733708 681.4056422554713
184791.5442310006 671.3169863364892
184129.56429834102 661.9799326595676
183476.45030728838 653.1139910526399
182832.69425792503 643.75604936335
182198.12359080344 634.5706671215885
181572.44568003732 625.6779107661278
180955.75635127057 616.6893287667481
180347.80444496276 607.9519063078042
179748.02064

126964.48393742164 136.1582428611291
126828.95320170095 135.53073572069115
126693.98535153434 134.96785016660579
126559.76786153528 134.21748999906413
126426.20735887242 133.56050266286184
126293.20466249908 133.00269637333986
126160.77972039438 132.42494210469886
126028.91730254126 131.86241785311722
125897.6538090394 131.26349350185774
125767.04750777167 130.60630126773322
125637.0707143909 129.97679338077432
125507.67801210807 129.3927022828284
125378.86869957468 128.8093125333835
125250.58249246643 128.28620710824907
125122.82259967888 127.75989278755151
124995.58974295399 127.23285672489146
124868.85878914293 126.73095381105668
124742.68038097359 126.17840816934768
124617.01622633816 125.66415463542216
124491.86060207254 125.15562426562246
124367.25535535262 124.6052467199188
124243.14947288725 124.10588246537372
124119.55652965163 123.59294323562062
123996.47553717165 123.08099247998325
123873.85075837807 122.62477879357175
123751.72536421647 122.12539416160143
123630.1876095368 

158890.21473591102 344.7846766328439
158548.5843956393 341.6303402717167
158209.72148429556 338.86291134374915
157873.67845353542 336.04303076013457
157540.3558580677 333.32259546773275
157209.7629728158 330.5928852518846
156881.9385426842 327.8244301316154
156556.86152465732 325.0770180268737
156234.54119501726 322.32032964006066
155915.06329954736 319.4778954698995
155598.13922060386 316.9240789434989
155283.78827400756 314.35094659629976
154972.11370229203 311.67457171552815
154662.81099371132 309.3027085807116
154355.9080543922 306.90293931911583
154051.39326683016 304.51478756204597
153749.6854929154 301.7077739147644
153450.5739975675 299.1114953478973
153153.73798665352 296.83601091397577
152859.22095067313 294.51703598038875
152567.08442718652 292.1365234866098
152277.35879764342 289.7256295430998
151989.74429107472 287.6145065687015
151704.38737980396 285.3569112707628
151421.1774840356 283.20989576834836
151140.09098668737 281.0864973482385
150860.97514357755 279.115843109815

118933.2377607721 101.17658250729437
118832.39229688798 100.84546388412127
118731.90718437149 100.48511251648597
118631.79358856668 100.11359580481076
118532.09145062797 99.70213793871517
498784.9633251135 498784.9633251135
474115.2211466542 24669.742178459303
452439.3967070544 21675.82443959982
433518.3186591819 18921.07804787252
417006.7870423503 16511.531616831606
402533.85146927973 14472.935573070543
389735.9269776484 12797.924491631333
378251.9715354567 11483.955442191684
367930.8601469101 10321.11138854659
358538.2474970631 9392.612649847055
349971.5954370588 8566.6520600043
342106.7570505595 7864.838386499265
334843.1567643528 7263.600286206696
328128.26466551237 6714.8920988404425
321883.43897629844 6244.8256892139325
316071.5106541543 5811.928322144144
310646.8350361777 5424.67561797658
305542.853780047 5103.9812561307335
300738.3431029532 4804.510677093756
296217.4334215195 4520.909681433695
291957.475172316 4259.9582492035115
287942.6055941946 4014.8695781214046
284146.60629

140137.64602477668 202.4450477779028
139936.42899012007 201.21703465661267
139736.43683016492 199.99215995514533
139537.61311050906 198.82371965586208
139340.142584288 197.47052622106276
139144.11334002725 196.02924426074605
138949.28119955029 194.83214047696674
138755.52177150716 193.75942804312217
138562.6970615301 192.8247099770524
138370.99113894068 191.7059225894336
138180.31311984852 190.67801909215632
137990.66074338133 189.65237646718742
137802.16670623474 188.49403714659275
137614.78423008297 187.38247615177534
137428.5676292945 186.21660078846617
137243.54800177462 185.01962751988322
137059.44989776087 184.09810401374125
136876.31445357186 183.13544418901438
136694.290445019 182.02400855286396
136513.39689120327 180.89355381572386
136333.71936219762 179.67752900565392
136155.1816985145 178.53766368312063
135977.62533322317 177.55636529132607
135800.99388340424 176.6314498189313
135625.34813378396 175.64574962027837
135450.5766798546 174.77145392936654
135276.70280604568 173.8

201130.62641368926 936.6498919849109
200210.71522492534 919.9111887639156
199308.43261848748 902.2826064378605
198422.57822607912 885.8543924083642
197552.99647623248 869.5817498466349
196698.1948262958 854.8016499366786
195858.4522090767 839.7426172191044
195033.2757353248 825.176473751897
194222.62147167494 810.6542636498634
193424.8153214923 797.8061501826451
192640.01948323534 784.7958382569486
191867.60244449598 772.4170387393679
191107.22331907033 760.3791254256503
190359.11912478437 748.1041942859592
189622.03296110878 737.0861636755872
188895.4505256514 726.5824354573851
188178.8118307234 716.6386949279986
187472.60684299504 706.2049877283571
186776.70094026008 695.9059027349576
186090.80889224578 685.8920480142988
185415.29263609851 675.516256147268
184749.43404035523 665.8585957432806
184092.9097178075 656.5243225477461
183446.14417124342 646.765546564071
182807.81086756437 638.3333036790427
182178.72745473403 629.0834128303395
181558.90425059432 619.8232041397132
180947.1941

127143.80465705298 136.5849412989919
127007.87149129601 135.933165756971
126872.58076845185 135.2907228441618
126737.94693695991 134.63383149194124
126603.86696075107 134.0799762088427
126470.37370726434 133.49325348672573
126337.48538315555 132.88832410878967
126205.18407509566 132.3013080598903
126073.48025916053 131.70381593513594
125942.41349582712 131.06676333340874
125811.91578457002 130.4977112570923
125681.96438063732 129.95140393270412
125552.57314736594 129.39123327138077
125423.72541980563 128.84772756030725
125295.39858284988 128.32683695574815
125167.61913484732 127.77944800256228
125040.35894076974 127.26019407757849
124913.6201641949 126.73877657484263
124787.37240909196 126.24775510294421
124661.62400673215 125.7484023598081
124536.40540122626 125.21860550588463
124411.6864168187 124.71898440756195
124287.48242520653 124.2039916121721
124163.86570091329 123.61672429324244
124040.7464002056 123.11930070769449
123918.12206624656 122.62433395902917
123796.11939446905 122.0

In [23]:
# obliczamy, w ilu probach wytrenowany model okazal sie lepszy od losowego
# przeprowadzamy test statystyczny - jak prawdopodobne jest to, by k pozytywnych prob bylo dzielem przypadku

def possibility_of_at_least_k_successes_in_n(k, n):
    p = 0.0
    # obliczamy kolejno prawdopodobienstwo k sukcesow, k+1 sukcesow, ...
    # przydadza Ci sie funkcje marh.comb() i math.pow()
    for i in range(k, n+1):
        p += math.comb(n, k) * math.pow(1/2, n)
    return p

p = 0.05
# metric = 'recall'
metric = 'precision'

# w ilu przypadkach okazalismy sie lepsi niz random?
positive_tests_count = 0
for i in range(len(results)):
    if results[i][metric] > random_results[i][metric]:
        positive_tests_count += 1

if possibility_of_at_least_k_successes_in_n(positive_tests_count, n_tests) <= p:
    print('We are better than random!')
else:
    print('There is no evidence we are better')

We are better than random!
